<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The Riemann Solution on $\tilde{S}_i$ using HLLE, in Curvilinear Coordinates
## Author: Patrick Nelson & Terrence Pierre Jacques

This notebook documents the function from the original `GiRaFFE` that calculates the flux for $\tilde{S}_i$ according to the method of [Harten, Lax, and von Leer](https://epubs.siam.org/doi/pdf/10.1137/1025002)  and [Einfeldt](https://epubs.siam.org/doi/10.1137/0725021) (HLLE), assuming that we have calculated the values of the velocity and magnetic field on the cell faces according to the piecewise-parabolic method (PPM) of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf), modified for the case of GRFFE. 

**Notebook Status:** <font color=green><b> Validated </b></font>

**Validation Notes:** This code has been self-validated against it's python module.

### NRPy+ Source Code for this module: 
* [CurviGiRaFFE_NRPy/CurviGiRaFFE_NRPy_Stilde_flux.py](../../edit/in_progress/CurviGiRaFFE_NRPy/CurviGiRaFFE_NRPy_Stilde_flux.py)
* [GiRaFFE_NRPy/GiRaFFE_NRPy_Characteristic_Speeds.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Characteristic_Speeds.py)

## Introduction

The differential equations that `CurviGiRaFFE` evolves are written in conservation form, and thus have two different terms that contribute to the time evolution of any conserved quantity $\mathcal{C}$, the flux term $\mathcal{F}$ and the source term $\mathcal{S}$:

$$
\partial_t \mathcal{C} + \partial_i \mathcal{F}^i = \mathcal{S} 
$$

We will use a high-resolution shock capturing scheme, which solves the Riemann problem using the HLL approximate Riemann solver to implement the above equation, to ensure that sharp features in our GRFFE fields are properly modeled.



We can then see that, if we rewrite this, the right-hand side (RHS) describing the time evolution $\partial_t \tilde{S}_i$ consists of two terms: the flux term and the source term. The flux term in particular can be tricky, as it may be discontinuous due to sharp features that may appear e.g., at current sheets or inside black hole horizons. If we were to simply approximate the term using finite-difference derivatives, such sharp features would lead to a [Gibbs-like phenomenon](https://en.wikipedia.org/wiki/Gibbs_phenomenon). So, we implement a different algorithm to take the derivative. Further, note that we need not take special care of the Christoffel symbols resulting from the covariant derivative of the flux; we can simply calculate it at the cell-centers along with the source term, thus it is not handled here. In this module we therefore handle

$$
\partial_t \tilde{\mathcal{S}}_j \propto \partial_i \left( \alpha \sqrt{\gamma} T^{i}_j \right).
$$



The flux term itself is, as written above, 

$$
\alpha \sqrt{\gamma} T^i_{j} = \alpha \sqrt{\gamma} g_{j \mu} T^{\mu i},
$$ 
where 

$$
T^{\mu \nu} = b^2 u^\mu u^\nu + \frac{1}{2} b^2 g^{\mu \nu} - b^\mu b^\nu.
$$

The C functions implemented in this notebook will compute the flux at a given point so that we can easily take its derivative later. Having reconstructed the values of $v^i_{(n)}$ and $B^i$ on the cell faces, we then compute the value of the flux of $\tilde{S}_i$ on each face using the [Harten, Lax, and von Leer](https://epubs.siam.org/doi/pdf/10.1137/1025002) and [Einfeldt](https://epubs.siam.org/doi/10.1137/0725021) (hereafter HLLE) approximate Riemann solver. In particular, for each component of $\tilde{S}_i$ in each direction, we compute the HLL flux as
$$
F^{\rm HLL} = \frac{c_{\rm min} f_{\rm R} + c_{\rm max} f_{\rm L} - c_{\rm min} c_{\rm max} (U_{\rm R}-U_{\rm L})}{c_{\rm min} + c_{\rm max}},
$$
where 
$$
f = \alpha \sqrt{\gamma} T^j_{i}
$$
and
$$
U = \tilde{S}_j.
$$
Here, $i$ is direction in which we are computing the Poynting flux, and $j$ is the component of the momentum we are computing it for. 

These two quantities are computed on both the left and right sides of each cell face. We will be able to draw heavily on the [GRFFE module](../../edit/GRFFE/equations.py) ([Tutorial](../Tutorial-GRFFE_Equations-Cartesian.ipynb)) and the [GRMHD module](../../edit/GRMHD/equations.py) ([Tutorial](../Tutorial-GRMHD_Equations-Cartesian.ipynb)) to compute $u^0$, $u^i$, and $b^\mu$, as well as the index-lowered forms of those vectors. Critically, these quantities depend on the Valencia 3-velocity $v^i_{(n)}$ and magnetic field $B^i$. We will not be using the usual gridfunctions for these, but rather the ones that we have previously calculated on the left and right sides of the cell faces using the [Piecewise Parabolic Method](Tutorial-GiRaFFE_NRPy_Ccode_library-PPM.ipynb).

The speeds $c_\min$ and $c_\max$ reflect the characteristic speeds of the plasma waves. In GRFFE, the expressions defining them reduce to a function of only the metric quantities: 

\begin{align}
c_\min &= - \min(c_-,0) \\ 
c_\max &= \max(c_+,0),
\end{align}
where
$$
c_\pm = \left. \left(-b \pm \sqrt{b^2-4ac}\right)\middle/ \left(2a\right) \right.,
$$
and
$$a = 1/\alpha^2,$$ 
$$b = 2 \beta^i / \alpha^2$$
and $$c = g^{ii} - (\beta^i)^2/\alpha^2.$$

These are coded in [Tutorial-GiRaFFE_NRPy-Characteristic_Speeds](Tutorial-GiRaFFE_NRPy-Characteristic_Speeds.ipynb).

Note that $c_\pm$ must be computed on each cell face, meaning that all the above metric quantities must be interpolated to cell faces. Metric face values are computed as described in [this notebook](Tutorial-GiRaFFE_NRPy-Metric_Face_Values.ipynb).

The algorithm for finite-volume methods in general is as follows: 

1. The Reconstruction Step - Piecewise Parabolic Method
    1. Within each cell, fit to a function that conserves the volume in that cell using information from the neighboring cells
        * PPM will automatically fit to parabolas in smooth regions, and lower-order polynomials in sharp regions
    1. Use that fit to define the state at the left and right interface of each cell
    1. Apply a slope limiter to mitigate Gibbs-like phenomena
1. Interpolate the value of the metric gridfunctions on the cell faces
1. **Solving the Riemann Problem - Harten, Lax, (This notebook, $\tilde{S}_i$ only)**
    1. **Use the left and right reconstructed states to calculate the unique state at boundary**
1. Use the unique state to estimate the derivative in the cell

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#s_i_flux): The $\tilde{S}_i$ function
    1. [Step 2.a](#fluxes): Compute the HLLE fluxes
1. [Step 3](#code_validation): Code Validation against `GiRaFFE_NRPy.Stilde_flux` NRPy+ Module
1. [Step 4](#derive_speed): Complete Derivation of the Wave Speeds
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code imports the core NRPy+ functionality after first adding the main NRPy+ directory to the path. 

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
GRMHD_dir_path = os.path.join("../GRMHD_formulation/")
sys.path.append(GRMHD_dir_path)
import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

nrpy_dir_path = os.path.join("../nrpy/nrpytutorial/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
    
from outputC import outputC, outCfunction, outC_function_dict # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

thismodule = "IGM-fluxes"

Ccodesdir = "IGM_standalone_Ccodes/"
cmd.mkdir(os.path.join(Ccodesdir))

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

CoordSystem = "Cartesian"

# Set coordinate system to dst_basis
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

<a id='s_i_flux'></a>

# Step 2: The $\tilde{S}_i$ function \[Back to [top](#toc)\]
$$\label{s_i_flux}$$


<a id='fluxes'></a>

## Step 2.a: Compute the HLLE fluxes \[Back to [top](#toc)\]
$$\label{fluxes}$$

Finally, we can compute the flux in each direction. This momentum flux in the $m$ direction is defined as $\alpha \sqrt{\gamma} T^m_{\ \ j}$, based on the input `flux_dirn`. We have already defined $\alpha \sqrt{\gamma}$, using the BSSN confomal factor $cf$, so all we need to do is calculate $T^m_{\ \ j}$, where $T^{\mu \nu} = b^2 u^\mu u^\nu + \frac{1}{2} b^2 g^{\mu \nu} - b^\mu b^\nu$. In doing this index-lowering operation, recall that $g^{\mu \nu} g_{\nu \alpha} = \delta^\mu_\alpha$. We will do so in accordance with the method published by [Harten, Lax, and von Leer](https://epubs.siam.org/doi/pdf/10.1137/1025002)  and [Einfeldt](https://epubs.siam.org/doi/10.1137/0725021) (hereafter HLLE) to solve the Riemann problem. So, we define $f(u) = T^m_{\ \ j}$ on each face as 
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^m) u_j + (P+\frac{1}{2}b^2) \delta^m_j - b^m b_j \right);
$$
Because $\rho = P = 0$ in GRFFE and $u^0 v^m = u^m$ in general (since $v^m$ is the drift velocity here), this simplifies to 
$$
f = \alpha \sqrt{\gamma} \left( b^2 u^m u_j + \frac{1}{2}b^2 \delta^m_j - b^m b_j \right).
$$
We use $j$ to correspond to the component of the flux we are calculating; that is, $j=0$ corresponds to $x$, and so forth (however, remember that in a NRPy+ 3-vector, the numbers will still run from 0 to 2). $\delta^i_j$ is the standard Kronecker delta. We also define $U_{\rm R}$ and $U_{\rm L}$:
$$
U = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 u_j - b^0 b_j \right),
$$
which, in GRFFE, simplifies to 
$$
U = \alpha \sqrt{\gamma} \left( b^2 u^0 u_j - b^0 b_j \right).
$$
In NRPy+, we'll let the GRMHD and GRFFE modules handle these.

and combine based on eq. 3.15 in the HLLE paper,
$$
F^{\rm HLL} = \frac{c_{\rm min} f_{\rm R} + c_{\rm max} f_{\rm L} - c_{\rm min} c_{\rm max} (U_{\rm R}-U_{\rm L})}{c_{\rm min} + c_{\rm max}},
$$

We'll write the HLLE step as a function so that we can loop over `flux_dirn` and `mom_comp` and write each version needed as we need it.

In this tutorial module we will compute the flux terms for the conservative variables $U=\left\{\rho_{\star},\tilde{\tau},\tilde{S}_{i}\right\}$, which are defined in terms of the primitive variables as

$$
\left(
\begin{matrix}
\rho_{\star}\\
\tilde{\tau}\\
\tilde{S}_{i}
\end{matrix}
\right)
=
\left(
\begin{matrix}
\alpha\sqrt{\gamma}\rho_{b}u^{0}\\
\alpha^{2}\sqrt{\gamma}T^{00}-\rho_{\star}\\
\left(\rho_{\star}h + \alpha u^{0}\sqrt{\gamma}b^{2}\right)u_{i} - \alpha\sqrt{\gamma}b^{0}b_{i}
\end{matrix}
\right)\ .
$$

The flux terms for these conservative variables are

$$
\boldsymbol{F} 
= 
\left(
\begin{matrix}
F^{i}_{\rho_{\star}}\\
F^{i}_{\tilde{\tau}}\\
\left(F_{\tilde{S}}\right)^{j}_{\ i}
\end{matrix}
\right)
=
\left(
\begin{matrix}
\rho_{\star}v^{i}\\
\alpha^{2}\sqrt{\gamma}T^{0j} - \rho_{\star}v^{j}\\
\alpha\sqrt{\gamma}T^{j}_{\ i}
\end{matrix}
\right)\ .
$$

The MHD flux algorithm computes, for each of the fluxes above, the standard Harten-Lax-van Leer (HLL) flux

$$
\boxed{F^{\rm HLL} = \frac{c^{-}F_{r} + c^{+}F_{l} - c^{+}c^{-}\left(U_{r} - U_{l}\right)}{c^{+} + c^{-}}}\ .
$$


In [2]:
# We'll rewrite this assuming that we've passed the entire reconstructed
# gridfunctions. You could also do this with only one point, but then you'd
# need to declare everything as a Cparam in NRPy+
    
def calculate_GRMHD_Tmunu_and_contractions(formalism, flux_dirn, mom_comp, 
                                          gammaDD,betaU,alpha,
                                          rho_b, P, h, u4U, BU):
    GRMHD.set_up_base_vars(formalism=formalism)
    
    GRMHD.compute_vU_from_u4U__no_speed_limit(u4U)
    
    # First compute stress-energy tensor T4UU and T4UD:
    GRMHD.compute_sqrtgammaDET(gammaDD)
    GRMHD.compute_smallb4U(gammaDD, betaU, alpha, u4U, BU, GRMHD.sqrt4pi)
    GRMHD.compute_smallbsquared(gammaDD, betaU, alpha, GRMHD.smallb4U)

    # First compute stress-energy tensor T4UU and T4UD:
    GRMHD.compute_T4UU(gammaDD,betaU,alpha, rho_b, P, h, u4U, GRMHD.smallb4U, GRMHD.smallbsquared)
    GRMHD.compute_T4UD(gammaDD,betaU,alpha, GRMHD.T4UU)
    
    # Compute conservative variables in terms of primitive variables
    GRMHD.compute_rho_star(alpha, GRMHD.sqrtgammaDET, rho_b,u4U)
    GRMHD.compute_tau_tilde(alpha, GRMHD.sqrtgammaDET, GRMHD.T4UU,GRMHD.rho_star)
    GRMHD.compute_S_tildeD(alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

    # Next compute fluxes of conservative variables
    GRMHD.compute_rho_star_fluxU(GRMHD.VU,GRMHD.rho_star)
    GRMHD.compute_tau_tilde_fluxU(alpha, GRMHD.sqrtgammaDET, GRMHD.VU, GRMHD.T4UU, GRMHD.rho_star)
    GRMHD.compute_S_tilde_fluxUD (alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

    global U_rho_star, F_rho_star
    U_rho_star = GRMHD.rho_star
    F_rho_star = GRMHD.rho_star_fluxU[flux_dirn]

    global U_tau_tilde, F_tau_tilde
    U_tau_tilde = GRMHD.tau_tilde
    F_tau_tilde = GRMHD.tau_tilde_fluxU[flux_dirn]

    global U_S_tilde, F_S_tilde
    U_S_tilde = GRMHD.S_tildeD[mom_comp]
    F_S_tilde = GRMHD.S_tilde_fluxUD[flux_dirn][mom_comp]

def HLLE_solver(cmax, cmin, Fr, Fl, Ur, Ul):
    # This solves the Riemann problem for the mom_comp component of the momentum
    # flux StildeD in the flux_dirn direction.

    # st_j_flux = (c_\min f_R + c_\max f_L - c_\min c_\max ( st_j_r - st_j_l )) / (c_\min + c_\max)
    return (cmin*Fr + cmax*Fl - cmin*cmax*(Ur-Ul) )/(cmax + cmin)


Finally, we write the function that computes the actual flux. We take the parameter `flux_dirn` as input, so we can eventually create one C file for each flux direction. In each file, we will include the math to calculate each momentum-flux component `mom_comp` in that direction by looping over `mom_comp`.

We have written the function `HLLE_solver()` so that we can easily compute the flux as specified by those two indices. Note that we've also rescaled the flux at the end of the calculation, such that it can safely finite difference it later on.

The characteristic wave speeds are calculated in the module [Tutorial-GiRaFFE_NRPy-Characteristic_Speeds](Tutorial-GiRaFFE_NRPy-Characteristic_Speeds.ipynb); we will simply import that here.

In [3]:
def calculate_HLLE_fluxes(formalism, flux_dirn, alpha_face, gamma_faceDD, beta_faceU,
                          u4_rU, u4_lU, B_rU, B_lU,
                          rho_b_r, rho_b_l,
                          P_r, P_l,
                          h_r, h_l,
                          cmin, cmax):

    global Stilde_flux_HLLED, rho_star_HLLE_flux, tau_tilde_HLLE_flux
    Stilde_flux_HLLED = ixp.zerorank1()
#     rescaled_Stilde_fluxD = ixp.zerorank1()
    for mom_comp in range(3):
        calculate_GRMHD_Tmunu_and_contractions(formalism, flux_dirn, mom_comp, 
                                               gamma_faceDD,beta_faceU,
                                               alpha_face,
                                               rho_b_r, P_r, h_r, u4_rU, 
                                               B_rU)
        
        F_S_tilde_r = F_S_tilde
        U_S_tilde_r = U_S_tilde
        
        if mom_comp==0:
            U_rho_star_r = U_rho_star
            F_rho_star_r = F_rho_star

            U_tau_tilde_r = U_tau_tilde
            F_tau_tilde_r = F_tau_tilde            
        
        calculate_GRMHD_Tmunu_and_contractions(formalism, flux_dirn, mom_comp, 
                                               gamma_faceDD,beta_faceU,
                                               alpha_face,
                                               rho_b_l, P_l, h_l, u4_lU, 
                                               B_lU)
        
        F_S_tilde_l = F_S_tilde
        U_S_tilde_l = U_S_tilde
        
        if mom_comp==0:
            U_rho_star_l = U_rho_star
            F_rho_star_l = F_rho_star

            U_tau_tilde_l = U_tau_tilde
            F_tau_tilde_l = F_tau_tilde
            
            # now calculate HLLE derived fluxes, and rescale all of them
            rho_star_HLLE_flux = HLLE_solver(cmax, cmin, 
                                      F_rho_star_r, F_rho_star_l, 
                                      U_rho_star_r, U_rho_star_l)
            
            tau_tilde_HLLE_flux = HLLE_solver(cmax, cmin, 
                                      F_tau_tilde_r, F_tau_tilde_l, 
                                      U_tau_tilde_r, U_tau_tilde_l)
        
        # Rescale the flux term, to be FD
        Stilde_flux_HLLED[mom_comp] = HLLE_solver(cmax, cmin, 
                                      F_S_tilde_r, F_S_tilde_l, 
                                      U_S_tilde_r, U_S_tilde_l)

There is some additional complexity to consider in generating the C code for these expressions. The flux term we need to add to the RHSs is a finite difference of the fluxes we have calculated so far, so these cannot be simple pointwise operations. However, we also cannot use NRPy+'s build-in finite-differencing tools because of how we store the reconstructed quantities (that is, quantities reconstructed on the $i-1/2$ face is stored at point $i$ in memory), which makes the FD template we need look just like a forward finite-differencing template, which NRPy+ cannot do. So, we must write the code to read and write data from and to memory ourselves. We will do so algorithmically with python string operations, so as to reduce the possibility from human error. 

We will loop over the interior, but use string replacement to include an extra point in the $+x,y,z$ ghostzones. The finite differences here will be done backwards from what we're used to; while we would normally read from several points and then write to a single point, we will read from a single point and then write to two points. In this particular case, this will reduce the number of times we recalculate things. 

In [4]:
# We will pass values of the gridfunction on the cell faces into the function. This requires us
# to declare them as C parameters in NRPy+. We will denote this with the _face infix/suffix.
formalism="BSSN"

sqrt4pi = sp.symbols("sqrt4pi")
alpha_face = sp.symbols("alpha_face")
if formalism=="BSSN":
    cf_face = sp.symbols("cf_face")
    h_faceDD = ixp.declarerank2("h_faceDD","sym01",DIM=3)
    vet_faceU = ixp.declarerank1("vet_faceU", DIM=3)
    beta_faceU = vet_faceU
    
    import BSSN.ADM_in_terms_of_BSSN as AitoB
    AitoB.ADM_in_terms_of_BSSN()

    import BSSN.BSSN_quantities as Bq

    gamma_faceDD = ixp.zerorank2()
    for i in range(3):
        for j in range(3):
            gamma_faceDD[i][j] = AitoB.gammaDD[i][j].subs(Bq.hDD[i][j], h_faceDD[i][j]).subs(Bq.cf, cf_face)

else:
    beta_faceU = ixp.declarerank1("beta_faceU") 
    gamma_faceDD = ixp.declarerank2("gamma_faceDD","sym01")
               
# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
u4_rU = ixp.declarerank1("u4_rU", DIM=4)
u4_lU = ixp.declarerank1("u4_lU", DIM=4)

B_rU = ixp.declarerank1("B_rU", DIM=4)
B_lU = ixp.declarerank1("B_lU", DIM=4)

cmin_dirn0 = sp.symbols("cmin_dirn0")
cmin_dirn1 = sp.symbols("cmin_dirn1")
cmin_dirn2 = sp.symbols("cmin_dirn2")

cmax_dirn0 = sp.symbols("cmax_dirn0")
cmax_dirn1 = sp.symbols("cmax_dirn1")
cmax_dirn2 = sp.symbols("cmax_dirn2")

cmins = [cmin_dirn0, cmin_dirn1, cmin_dirn2]
cmaxs = [cmax_dirn0, cmax_dirn1, cmax_dirn2]

h_r = sp.symbols("h_r")
h_l = sp.symbols("h_l")

P_r = sp.symbols("P_r")
P_l = sp.symbols("P_l")

rho_b_r = sp.symbols("rhob_r")
rho_b_l = sp.symbols("rhob_l")

Note that since we are calculating HLLE fluxes on cell-interfaces, we must also define the scale factors in the same places. Thus, we make use of the Kronecker delta, defined in our code as 

```C
const int kronecker_delta[4][3] = { { 0,0,0 },
                                    { 1,0,0 },
                                    { 0,1,0 },
                                    { 0,0,1 } };
```

In [5]:
prims_velocities = ["u4_rU0", "u4_rU1", "u4_rU2", "u4_rU3",
                    "u4_lU0", "u4_lU1", "u4_lU2", "u4_lU3"]

prims_mag_field = ["B_rU0", "B_rU1", "B_rU2",
                   "B_lU0", "B_lU1", "B_lU2"]

other_prims = ["P", "h", "rhob"]

params = ["GAMMA_SPEED_LIMIT", "TINYDOUBLE", "sqrt4pi"]

prestring = ""

for var in prims_velocities + prims_mag_field:
    prestring += "const double "+str(var)+" = reconstructed_prims->"+str(var)+";\n"

for var in other_prims:
    prestring += "const double "+str(var)+"_r = reconstructed_prims->"+str(var)+"_r;\n"
    prestring += "const double "+str(var)+"_l = reconstructed_prims->"+str(var)+"_l;\n"
for var in params:
    prestring += "const double "+str(var)+" = rhss_params->"+str(var)+";\n"

prestring += "const double "+str(alpha_face)+" = metric_face_quantities->"+str(alpha_face)+";\n"

if formalism=="BSSN":
    prestring += "const double "+str(cf_face)+" = metric_face_quantities->"+str(cf_face)+";\n"

    for i in range(3):
        vetU_var = vet_faceU[i]
        prestring += "const double "+str(vetU_var)+" = metric_face_quantities->"+str(vetU_var)+";\n"

    for i in range(3):
        for j in range(3):
            hDD_var = h_faceDD[i][j]
            prestring += "const double "+str(hDD_var)+" = metric_face_quantities->"+str(hDD_var)+";\n"

else:
    for i in range(3):
            betaU_var = beta_faceU[i]
            prestring += "const double "+str(betaU_var)+" = metric_face_quantities->"+str(betaU_var)+";\n"

    for i in range(3):
        for j in range(3):
            gammaDD_var = gamma_faceDD[i][j]
            prestring += "const double "+str(gammaDD_var)+" = metric_face_quantities->"+str(gammaDD_var)+";\n"

In [6]:
outCparams = "outCverbose=False,CSE_sorting=True"
# write_cmax_cmin=True

vars_to_write = ["conservative_fluxes->StildeD0_rhs", "conservative_fluxes->StildeD1_rhs", "conservative_fluxes->StildeD2_rhs", 
                 "conservative_fluxes->rho_star_rhs", "conservative_fluxes->tau_tilde_rhs"]

c_type = "void"
params   = "const rhss_paramstruct *restrict rhss_params, "
params  += "const prims_struct *restrict reconstructed_prims, "
params  += "const metric_quantities_struct *restrict metric_face_quantities, "
params  += "conservative_fluxes_struct *restrict conservative_fluxes"

calc_char_speeds_params_str = "(rhss_params, reconstructed_prims, metric_face_quantities, conservative_fluxes)"

for flux_dirn in range(3):
    var = cmins[flux_dirn]
    cmin_str = "const double "+str(var)+" = conservative_fluxes->"+str(var)+";\n"

    var = cmaxs[flux_dirn]
    cmax_str = "const double "+str(var)+" = conservative_fluxes->"+str(var)+";\n"
    
    calc_char_speeds_func_str = "calculate_characteristic_speed_"+str(flux_dirn)+"th_direction"

    calculate_HLLE_fluxes(formalism, flux_dirn, alpha_face, gamma_faceDD, beta_faceU,
                          u4_rU, u4_lU, B_rU, B_lU,
                          rho_b_r, rho_b_l,
                          P_r, P_l,
                          h_r, h_l,
                          cmins[flux_dirn], cmaxs[flux_dirn])
    
    vars_rhs = [Stilde_flux_HLLED[0], 
                Stilde_flux_HLLED[1], 
                Stilde_flux_HLLED[2], 
                rho_star_HLLE_flux,
                tau_tilde_HLLE_flux]

    body = outputC(vars_rhs, vars_to_write, params=outCparams, 
                   filename="returnstring", prestring=(calc_char_speeds_func_str+
                                                       calc_char_speeds_params_str+";\n\n"+
                                                       cmin_str+cmax_str+prestring))

    desc = "Compute the HLLE-derived fluxes on the left face in the " + str(flux_dirn) + "direction for all components."
    name = "calculate_HLLE_fluxes" + str(flux_dirn)
    
    outCfunction(
    outfile=os.path.join(Ccodesdir,name+".h"),
#     includes=includes,
    desc=desc,
    name=name,
    params=params,
    body= body, 
    enableCparameters=False)

Output C function calculate_HLLE_fluxes0() to file IGM_standalone_Ccodes/calculate_HLLE_fluxes0.h
Output C function calculate_HLLE_fluxes1() to file IGM_standalone_Ccodes/calculate_HLLE_fluxes1.h
Output C function calculate_HLLE_fluxes2() to file IGM_standalone_Ccodes/calculate_HLLE_fluxes2.h


<a id='code_validation'></a>

# Step 3:  Code Validation against `GiRaFFE_NRPy.Stilde_flux` NRPy+ Module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the C code generated by 
1. this tutorial and 
2. the NRPy+ [CurviGiRaFFE_NRPy.CurviGiRaFFE_NRPy_Stilde_flux](../../edit/in_progress/CurviGiRaFFE_NRPy.CurviGiRaFFE_NRPy_Stilde_flux.py) module.

In [7]:
files = ["calculate_HLLE_fluxes0.h",
         "calculate_HLLE_fluxes1.h",
         "calculate_HLLE_fluxes2.h"]

names = ["calculate_HLLE_fluxes0",
         "calculate_HLLE_fluxes1",
         "calculate_HLLE_fluxes2"]

In [8]:
# Define the directory that we wish to validate against:
valdir = "IGM_Ccode_library/"
cmd.mkdir(os.path.join(valdir))

import IGM_All_fluxes as fluxes
fluxes.add_to_Cfunction_dict__GRMHD_fluxes(formalism=formalism,
                                           outCparams=outCparams)

for name in names:
    with open(os.path.join(valdir,name+".h"), "w") as file:
        file.write(outC_function_dict[name])

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .h file. See differences above.")
            sys.exit(1)

Printing difference between original C code and this code...
Checking file calculate_HLLE_fluxes0.h
No difference. TEST PASSED!
Checking file calculate_HLLE_fluxes1.h
No difference. TEST PASSED!
Checking file calculate_HLLE_fluxes2.h
No difference. TEST PASSED!


<a id='derive_speed'></a>

# Step 4: Complete Derivation of the Wave Speeds \[Back to [top](#toc)\]
$$\label{derive_speed}$$

This computes phase speeds in the direction given by flux_dirn. Note that we replace the full dispersion relation with a simpler one, which overestimates the maximum speeds by a factor of ~2. See full discussion around Eqs. 49 and 50 in [Duez, et al.](http://arxiv.org/pdf/astro-ph/0503420.pdf). In summary, we solve the dispersion relation (in, e.g., the $x$-direction) with a wave vector of $k_\mu = (-\omega,k_x,0,0)$. So, we solve the approximate dispersion relation $\omega_{\rm cm}^2 = [v_A^2 + c_s^2 (1-v_A^2)]k_{\rm cm}^2$ for the wave speed $\omega/k_x$, where the sound speed $c_s = \sqrt{\Gamma P/(h \rho_0)}$, the Alfv&eacute;n speed $v_A = 1$ (in GRFFE), $\omega_{\rm cm} = -k_\mu k^\mu$ is the frequency in the comoving frame, $k_{\rm cm}^2 = K_\mu K^\mu$ is the wavenumber squared in the comoving frame, and $K_\mu = (g_{\mu\nu} + u_\mu u_\nu)k^\nu$ is the part of the wave vector normal to the four-velocity $u^\mu$. See below for a complete derivation.

What follows is a complete derivation of the quadratic we solve. We start from the following relations:
\begin{align}
w_{\rm cm} &= (-k_0 u^0 - k_x u^x) \\
k_{\rm cm}^2 &= K_{\mu} K^{\mu}, \\
K_{\mu} K^{\mu} &= (g_{\mu a} + u_{\mu} u_a) k^a  g^{\mu b} [ (g_{c b} + u_c u_b) k^c ] \\
\end{align}
The last term of the above can be written as follow:
$$
(g_{c b} + u_{c} u_{b}) k^c = (\delta^{\mu}_c + u_c u^{\mu} ) k^c
$$

Then,
\begin{align}
K_{\mu} K^{\mu} &= (g_{\mu a} + u_{\mu} u_a) k^a  g^{\mu b} [ (g_{c b} + u_c u_b) k^c ] \\
                 &= (g_{\mu a} + u_{\mu} u_a) k^a  (\delta^{\mu}_c + u_c u^{\mu} ) k^c \\
                 &=[(g_{\mu a} + u_{\mu} u_a) \delta^{\mu}_c + (g_{\mu a} + u_{\mu} u_a) u_c u^{\mu} ] k^c k^a \\
                 &=[(g_{c a} + u_c u_a) + (u_c u_a -  u_a u_c] k^c k^a \\
                 &=(g_{c a} + u_c u_a) k^c k^a \\
                 &= k_a k^a + u^c u^a k_c k_a \\
k^a = g^{\mu a} k_{\mu} &= g^{0 a} k_0 + g^{x a} k_x \\
k_a k^a &= k_0 g^{0 0} k_0 + k_x k_0 g^{0 x} + g^{x 0} k_0 k_x + g^{x x} k_x k_x \\
         &= g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2 \\
u^c u^a k_c k_a &= (u^0 k_0 + u^x k_x) (u^0 k_0 + u^x k_x) = (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 \\
(k_0 u^0)^2  + 2 k_x u^x k_0 u^0 + (k_x u^x)^2 &= v_0^2 [ (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 + g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2] \\
(1-v_0^2) (u^0 k_0 + u^x k_x)^2 &= v_0^2 (g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2) \\
(1-v_0^2) (u^0 k_0/k_x + u^x)^2 &= v_0^2 (g^{00} (k_0/k_x)^2 + 2 g^{x0} k_0/k_x + g^{xx}) \\
(1-v_0^2) (u^0 X + u^x)^2 &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
(1-v_0^2) ((u^0)^2 X^2 + 2 u^x (u^0) X + (u^x)^2) &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
0 &= X^2 ( (1-v_0^2) (u^0)^2 - v_0^2 g^{00}) + X (2 u^x u^0 (1-v_0^2) - 2 v_0^2 g^{x0}) + (1-v_0^2) (u^x)^2 - v_0^2 g^{xx} \\
a &= (1-v_0^2) (u^0)^2 - v_0^2 g^{00} = (1-v_0^2) (u^0)^2 + v_0^2/\alpha^2 \leftarrow {\rm VERIFIED} \\
b &= 2 u^x u^0 (1-v_0^2) - 2 v_0^2 \beta^x/\alpha^2 \leftarrow {\rm VERIFIED,\ } X\rightarrow -X, {\rm because\ } X = -w/k_1, {\rm \ and\ we\ are\ solving\ for} -X. \\
c &= (1-v_0^2) (u^x)^2 - v_0^2 (\gamma^{xx}\psi^{-4} - (\beta^x/\alpha)^2) \leftarrow {\rm VERIFIED} \\
v_0^2 &= v_A^2 + c_s^2 (1 - v_A^2) \\
\end{align}

<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy-Stilde-flux.pdf](Tutorial-GiRaFFE_NRPy-Stilde-flux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [9]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy-Stilde-flux",location_of_template_file=os.path.join(".."))

[NbConvertApp] WARNING | pattern 'Tutorial-GiRaFFE_NRPy-Stilde-flux.ipynb' matched no files
Created Tutorial-GiRaFFE_NRPy-Stilde-flux.tex, and compiled LaTeX file to
    PDF file Tutorial-GiRaFFE_NRPy-Stilde-flux.pdf
